In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 1.3 MB/s eta 0:00:0000:0100:01


In [ ]:
# Zadanie 1

import pulp as pl
import numpy as np
from scipy.stats import multivariate_t


# Print generated scenarios
print("Generated Price Scenarios:")
# print(scenarios)

print("==============================================================================")

# Define the problem
model = pl.LpProblem("Maximize_Profit", pl.LpMaximize)

# Products and days
products = ['Chleb_domowy_0_5kg', 'Chleb_domowy_0_9kg', 'Chleb_na_zakwasie', 'Bulki', 'Bagietki', 'Rogaliki_z_czekolada']
days = ['Pon', 'Wt', 'Sr', 'Czw', 'Pt', 'Sob']

# Income from product sales in PLN/piece
product_income = {'Chleb_domowy_0_5kg': 5.2, 'Chleb_domowy_0_9kg': 7.9, 'Chleb_na_zakwasie': 6.2, 'Bulki': 1.4, 'Bagietki': 1.55, 'Rogaliki_z_czekolada': 2.1}

# Time available per month (in hours)
# days_per_day = 24
shifts_per_day = 2
hours_per_shift = 8
hours_per_day = shifts_per_day * hours_per_shift

# Initial stock and storage cost
initial_stock = 2
storage_cost_per_unit_per_day = 1
storage_capacity = 100
end_stock = 0

# Production times required per product in hours
production_times = {
    'Chleb_domowy_0_5kg': {'Dzieza': 4.7, 'Wazenie': 0.29, 'Dzielarka': 0.0, 'Tabula': 0.7, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 1.0, 'Piec': 0.68, 'Pomieszczenie_do_studzenia': 1.8, 'Pakowanie': 0.56},
    'Chleb_na_zakwasie': {'Dzieza': 8.6, 'Wazenie': 0.22, 'Dzielarka': 0.0, 'Tabula': 0.6, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 1.4, 'Piec': 1.0, 'Pomieszczenie_do_studzenia': 2.3, 'Pakowanie': 0.6},
    'Chleb_domowy_0_9kg': {'Dzieza': 5, 'Wazenie': 0.2, 'Dzielarka': 0.0, 'Tabula': 0.55, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 1.5, 'Piec': 1.2, 'Pomieszczenie_do_studzenia': 2.5, 'Pakowanie': 0.57},
    'Bulki': {'Dzieza': 1, 'Wazenie': 0.0, 'Dzielarka': 0.23, 'Tabula': 0.34, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 0.38, 'Piec': 0.25, 'Pomieszczenie_do_studzenia': 0.48, 'Pakowanie': 0.0},
    'Bagietki': {'Dzieza': 0.95, 'Wazenie': 0.0, 'Dzielarka': 0.24, 'Tabula': 0.0, 'Rogalikarka': 0.28, 'Nadziewarka': 0.0, 'Garownia': 0.33, 'Piec': 0.23, 'Pomieszczenie_do_studzenia': 0.42, 'Pakowanie': 0.0},
    'Rogaliki_z_czekolada': {'Dzieza': 0.55, 'Wazenie': 0.0, 'Dzielarka': 0.28, 'Tabula': 0.0, 'Rogalikarka': 0.31, 'Nadziewarka': 0.4, 'Garownia': 0.25, 'Piec': 0.18, 'Pomieszczenie_do_studzenia': 0.82, 'Pakowanie': 0.42}
}

# Market limits for sales of Products
market_limits = {
    "Chleb_domowy_0_5kg": {"Pon": 200, "Wt": 300, "Sr": 0, "Czw": 200, "Pt": 300, "Sob": 0},
    "Dzieza": {"Pon": 200, "Wt": 300, "Sr": 0, "Czw": 200, "Pt": 300, "Sob": 0},
    "Wazenie": {"Pon": 200, "Wt": 300, "Sr": 0, "Czw": 200, "Pt": 300, "Sob": 0},
    "Tabula": {"Pon": 200, "Wt": 300, "Sr": 0, "Czw": 200, "Pt": 300, "Sob": 0},
    "Rogalikarka": {"Pon": 200, "Wt": 300, "Sr": 0, "Czw": 200, "Pt": 300, "Sob": 0},
    "Nadziewarka": {"Pon": 200, "Wt": 300, "Sr": 0, "Czw": 200, "Pt": 300, "Sob": 0},
    "Garownia": {"Pon": 200, "Wt": 300, "Sr": 0, "Czw": 200, "Pt": 300, "Sob": 0},
    "Piec": {"Pon": 200, "Wt": 300, "Sr": 0, "Czw": 200, "Pt": 300, "Sob": 0},
    "Pomieszczenie_do_studzenia": {"Pon": 200, "Wt": 300, "Sr": 0, "Czw": 200, "Pt": 300, "Sob": 0},
    "Pakowanie": {"Pon": 200, "Wt": 300, "Sr": 0, "Czw": 200, "Pt": 300, "Sob": 0},
}

# Machines available
machines = {"Dzieza": 4, "Wazenie": 2, "Dzielarka": 3, "Tabula": 1, "Rogalikarka": 1, "Nadziewarka": 1, "Garownia": 1, "Piec": 2, "Pomieszczenie_do_studzenia": 1, "Pakowanie": 1}

# Variables for production, sales, and storage of products each day
production_vars = pl.LpVariable.dicts("Production", ((product, day) for product in products for day in days), lowBound=0, cat='Integer')
sales_vars = pl.LpVariable.dicts("Sales", ((product, day) for product in products for day in days), lowBound=0, cat='Integer')
storage_vars = pl.LpVariable.dicts("Storage", ((product, day) for product in products for day in days), lowBound=0, upBound=storage_capacity, cat='Integer')

# Constrain for P4 sales based on P1 and P2 sales
# for day in days:
#   model += sales_vars['P4', day] >= sales_vars['P1', day] + sales_vars['P2', day]

# Inventory and sales constraints
for product in products:
    model += storage_vars[product, 'Pon'] == initial_stock + production_vars[product, 'Pon'] - sales_vars[product, 'Pon']  # for Pon
    for i in range(1, len(days)):
        day = days[i]
        prev_day = days[i-1]
        model += storage_vars[product, day] == storage_vars[product, prev_day] + production_vars[product, day] - sales_vars[product, day]  # for rest of the days
    model += storage_vars[product, 'Sob'] == end_stock  # Ending stock

# Machine time constraints for each product and day
for day in days:
    total_production_time = sum(production_vars[product, day] * production_times[product][machine]
                                for product in products
                                for machine in machines)
    model += total_production_time <= hours_per_day, f"Global_production_time_{day}"

# Market sales constraints
for product in products:
    for day in days:
        model += sales_vars[product, day] <= market_limits[product][day], f"Max_sales_{product}_{day}"
        model += sales_vars[product, day] <= production_vars[product, day] + (storage_vars[product, days[days.index(day) - 1]] if day != 'Jan' else initial_stock)


# Objective function: Maximize profit from sales minus storage costs
model += pl.lpSum([sales_vars[product, day] * product_income[product] - storage_vars[product, day] * storage_cost_per_unit_per_day
                   for product in products for day in days]), "Total_Profit"

# Solve the model
model.solve()

# Check the status of the solution and print the results
print("Status:", pl.LpStatus[model.status])
if pl.LpStatus[model.status] == 'Optimal':
    print("Total Profit:", pl.value(model.objective))

    # Printing headers
    print(f"{'Product':<10} {'day':<10} {'Production':<12} {'Sales':<10} {'Storage':<10}")

    # Display the results in tabular form
    for product in products:
        for day in days:
            production = production_vars[product, day].varValue
            sales = sales_vars[product, day].varValue
            storage = storage_vars[product, month].varValue
            print(f"{product:<10} {month:<10} {production:<12} {sales:<10} {storage:<10}")

    print("\nProduction Time Usage per Product and Month:")
    for month in days:
        print(f"\nMonth: {month}")
        for product in products:
            total_production_time = sum(
                production_vars[product, month].varValue * production_times[product].get(machine, 0)
                for machine in machines.keys()
            )
            print(f"  Product: {product}, Total Production Time: {total_production_time} hours")

else:
    print("No optimal solution was found.")

Generated Price Scenarios:


KeyError: 'Chleb_domowy_0_9kg'